# STATIC LAYERS
- This notebook static products by running ISCE3's `static.py` by taking a region as input.
- Uses the `isce3_src` kernel (created using [Create_Environments.ipynb](https://github.com/isce-framework/sds-ondemand/blob/main/environments/Create_Environments.ipynb)).
- Can be ran locally (preferably on a GPU instance), or as a PCM job.

# Parameters
This cell is marked `parameters`, indicating the variables within can substituted when running this notebook via `papermill`.
- `dem_vrt_s3_url`: S3 url to the DEM file to download.
- `watermask_vrt_s3_url`: S3 url to the DEM file to download.
- `gpu_enabled`: `1` to run using the GPU, `0` to use CPU instead. **Keep in mind that while disabling the GPU processing allows this notebook to be ran on an instance without a GPU, that does not guarantee the instance this notebook is running on is a non-GPU instance.** To run on a non-GPU instance on PCM, submit the job to a CPU-only queue.
- `config_yml`: The runconfig passed to `static.py`.

### Upload parameters (PCM only)
- `timestamp`: A time-string of the format `%Y%m%dT%H%M%S` indicating the time at which the job was submitted. This helps `pcm.py` find where the results of this job will be submitted when this notebook is ran as a PCM job.

In [ ]:
dem_vrt_s3_url = 's3://nisar-st-data-ondemand/zhan/watermask.vrt'
watermask_vrt_s3_url = 's3://nisar-st-data-ondemand/zhan/dem.vrt'
orbit_xml = '/home/jovyan/alos-to-insar/data/track34_frame72_west_himalayas/NISAR_ANC_L_PR_FOE_20250424T071519_20230105T103711_20230105T163746.xml'
pointing_xml = '/home/jovyan/alos-to-insar/data/track34_frame72_west_himalayas/NISAR_ANC_L_PR_FRP_20250424T071519_20230105T103711_20230105T163746.xml'
config_yml = '/home/jovyan/alos-to-insar/data/track34_frame72_west_himalayas/static.yml' # string
timestamp = '20240131T0123456' # string

# hysds specifications
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = '15GB'
_submission_type = 'iteration'
_label = 'STATIC LAYER PGE'

In [ ]:
# TODO - REMOVE THIS!!!!!!!!!!!!!
# with open(orbit_xml, 'r') as fp:
#     orbit_xml = fp.read()

# with open(pointing_xml, 'r') as fp:
#     pointing_xml = fp.read()

# with open(config_yml, 'r') as fp:
#     config_yml = fp.read()

In [ ]:
import os
import yaml
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PRODUCT_DIR, exist_ok=True)

if config_yml == '':
    with open(os.path.join(ISCE3_BUILD_DIR, 'share', 'nisar', 'defaults', 'static.yaml'), 'r') as f:
        CONFIG_YML = yaml.safe_load(f)
else:
    print('Using custom static.py run config...')
    CONFIG_YML = yaml.safe_load(config_yml)

yml_path = os.path.join(OUTPUT_DIR, 'static_final.yaml')

with open(os.path.join(OUTPUT_DIR, 'orbit_final.xml'), 'w', encoding='utf-8') as fp:
    fp.write(orbit_xml)

with open(os.path.join(OUTPUT_DIR, 'pointing_final.xml'), 'w', encoding='utf-8') as fp:
    fp.write(pointing_xml)

print(WORKING_DIR)

In [ ]:
import isce3_config
import yaml

# Grab the bounding box for the DEM and water mask
bbox = isce3_config.BoundingBox.load_from_geogrid(
    CONFIG_YML['runconfig']['groups']['processing']['geo_grid'])

print(bbox)


# Download the DEM locally
if isinstance(dem_vrt_s3_url, str) and dem_vrt_s3_url != '':
    dem_vrt_f = isce3_config.download_dem(dem_vrt_s3_url, DOWNLOAD_DIR)
else:
    print(f'DEM input ({dem_vrt_s3_url}) was not specified, downloading DEM via stage_dem.py instead.')

    dem_vrt_f = os.path.join(PRODUCT_DIR, 'dem.vrt')
    dem_f = os.path.join(PRODUCT_DIR, 'dem_0.tiff')
    
    # Stage the DEM using the combined bounds for dem_tiff_f
    _cmd_stage_dem = f'mamba run -n isce3_src --live-stream python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_dem.py -b {bbox.west} {bbox.south} {bbox.east} {bbox.north} -o {dem_vrt_f}'
    print(f'Executing:\n    {_cmd_stage_dem}')
    !{_cmd_stage_dem}


# Download the Watermask locally
if isinstance(watermask_vrt_s3_url, str) and watermask_vrt_s3_url != '':
    watermask_vrt_f = isce3_config.download_dem(watermask_vrt_s3_url, DOWNLOAD_DIR)
else:
    watermask_vrt_f = os.path.join(PRODUCT_DIR, 'WATERMASK.vrt')
    watermask_f = os.path.join(PRODUCT_DIR, 'WATERMASK_0.tiff')
    
    # Stage the DEM using the combined bounds for watermask_tiff_f
    _cmd_stage_watermask = f'mamba run -n isce3_src --live-stream python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_watermask.py -b {bbox.west} {bbox.south} {bbox.east} {bbox.north} -o {watermask_vrt_f}'
    print(f'Executing:\n    {_cmd_stage_watermask}')
    !{_cmd_stage_watermask}


isce3_config.write_static_config(CONFIG_YML, dem_vrt_f, watermask_vrt_f, yml_path)

!mamba run -n isce3_src --live-stream python -m nisar.workflows.static {yml_path}

In [5]:
import h5py
import json

# Grab some appropriate values for PCM to automatically stage out
# try:
#     h5_file = h5py.File(output_f, 'r')
#     polarization = h5_file['science']['LSAR']['RSLC']['swaths']['frequencyA']['listOfPolarizations'][0].decode('utf-8')
# except Exception as e:
#     print(f'Exception occured: {e}')
#     print(f'As the polarization was unable to be automatically detected, the polarization will be listed as \'HH\' by default.')
polarization = 'HH'
regex_name = isce3_config.RSLC_FORMAT.format(
    polarization=f'{polarization}00',
    timestamp=timestamp)
auto_output_dir = os.path.join(WORKING_DIR, regex_name)

# Create the automatic stage-out directory
!mkdir -p {auto_output_dir}
!cp {WORKING_DIR}/*-output.ipynb {auto_output_dir}
!cp {WORKING_DIR}/*.txt {auto_output_dir}
!cp {WORKING_DIR}/*.json {auto_output_dir}
!mv {yml_path} {auto_output_dir}
#!mv {output_f} {auto_output_dir}

# Move the new path variables to avoid confusion
yml_path = os.path.join(auto_output_dir, os.path.basename(yml_path))
output_f = os.path.join(auto_output_dir, os.path.basename(output_f))

# Create metadata files for the automatic stage-out directory
with open(os.path.join(auto_output_dir, f'{regex_name}.met.json'), 'w', encoding='utf-8') as f:
    content = {
        'polarization': polarization,
    }
    f.write(json.dumps(content))
with open(os.path.join(auto_output_dir, f'{regex_name}.dataset.json'), 'w', encoding='utf-8') as f:
    content = {
         'version': 'v1.0',
         'label': 'This is purely an EXAMPLE metadata file, the values in this file are not representative of this product.',
         'location': {
           'type': 'polygon',
           'coordinates': [
             [
                [-122.9059682940358,40.47090915967475],
                [-121.6679748715316,37.84406528996276],
                [-120.7310161872557,38.28728069813177],
                [-121.7043611684245,39.94137004454238],
                [-121.9536916840953,40.67097860759095],
                [-122.3100379696548,40.7267890636145],
                [-122.7640648263371,40.5457010812299],
                [-122.9059682940358,40.47090915967475]
              ]
            ]
        },
        'starttime': '2017-01-01T00:00:00',
        'endtime': '2017-01-01T00:05:00',
    }
    f.write(json.dumps(content))

NameError: name 'output_f' is not defined